In [1]:
# import os
# import re
# import glob
# from datetime import datetime
# import pandas as pd
#
# # --- Paths ---
# SRC_DIR  = r"C:\Users\leere\PycharmProjects\Football_ML3\Goals\2H_goal\ht_scoreline"
# DEST_DIR = os.path.join(SRC_DIR, "best_models_by_ht_scoreline")
# os.makedirs(DEST_DIR, exist_ok=True)
#
# # --- Filename patterns supported ---
# # Singles:
# # 1) model_metrics_'0-0'_YYYYMMDD_XXXXXXXX.csv
# # 2) model_metrics_('0-0',)_YYYYMMDD_XXXXXXXX.csv
# # 3) model_metrics_0-0_YYYYMMDD_XXXXXXXX.csv
# # Combined:
# # 4) model_metrics_'0-0'_COMBINED_YYYYMMDD_XXXXXXXX.csv
# # 5) model_metrics_('0-0',)_COMBINED_YYYYMMDD_XXXXXXXX.csv
# # 6) model_metrics_0-0_COMBINED_YYYYMMDD_XXXXXXXX.csv
# PATTERNS = [
#     re.compile(r"^model_metrics_'([^']+)'_(\d{8})_(\d{6})\.csv$"),
#     re.compile(r"^model_metrics_\('([^']+)',\)_(\d{8})_(\d{6})\.csv$"),
#     re.compile(r"^model_metrics_([^_]+)_(\d{8})_(\d{6})\.csv$"),
#     re.compile(r"^model_metrics_'([^']+)'_COMBINED_(\d{8})_(\d{6})\.csv$"),
#     re.compile(r"^model_metrics_\('([^']+)',\)_COMBINED_(\d{8})_(\d{6})\.csv$"),
#     re.compile(r"^model_metrics_([^_]+)_COMBINED_(\d{8})_(\d{6})\.csv$"),
# ]
#
# # Known parameter columns used for de-duplication
# XGB_PARAM_COLS = [
#     "n_estimators", "max_depth", "learning_rate", "min_child_weight",
#     "subsample", "colsample_bytree", "reg_lambda",
# ]
# MLP_PARAM_COLS = [
#     "hidden_layer_sizes", "alpha", "learning_rate_init",
#     "batch_size", "max_iter",
# ]
# DEDUP_KEYS = XGB_PARAM_COLS + MLP_PARAM_COLS + ["threshold"]
#
# # ---------- NEW: delete *_FAILED files ----------
# def is_failed_filename(path: str) -> bool:
#     base = os.path.basename(path)
#     name_no_ext, _ = os.path.splitext(base)
#     return base.endswith("_FAILED") or base.endswith("_FAILED.csv") or name_no_ext.endswith("_FAILED")
#
# def purge_failed_files(dirs: list[str]) -> None:
#     for d in dirs:
#         pattern = os.path.join(d, "model_metrics_*")
#         for p in glob.glob(pattern):
#             if os.path.isfile(p) and is_failed_filename(p):
#                 try:
#                     os.remove(p)
#                     print(f"✂ Deleted FAILED file: {p}")
#                 except Exception as e:
#                     print(f"[WARN] Could not delete FAILED file {p}: {e}")
# # ------------------------------------------------
#
# def extract_scoreline(filename: str) -> str | None:
#     base = os.path.basename(filename)
#     for rx in PATTERNS:
#         m = rx.match(base)
#         if m:
#             return m.group(1)  # scoreline
#     return None
#
# def files_by_scoreline(base_dir: str) -> dict[str, list[str]]:
#     out: dict[str, list[str]] = {}
#     for path in glob.glob(os.path.join(base_dir, "model_metrics_*.csv")):
#         # Skip any lingering *_FAILED files defensively
#         if is_failed_filename(path):
#             continue
#         sl = extract_scoreline(os.path.basename(path))
#         if not sl:
#             continue
#         out.setdefault(sl, []).append(path)
#     return out
#
# def read_csv_safely(path: str) -> pd.DataFrame:
#     try:
#         return pd.read_csv(path, low_memory=False)
#     except Exception as e:
#         print(f"[WARN] Failed to read {path}: {e}")
#         return pd.DataFrame()
#
# def _coerce_bool_series(s: pd.Series) -> pd.Series:
#     if s.dtype == bool:
#         return s
#     # numeric (1/0) -> True/False
#     try:
#         sn = pd.to_numeric(s, errors="coerce")
#         if sn.notna().any():
#             return (sn.fillna(0) != 0)
#     except Exception:
#         pass
#     # string interpretation
#     truthy = {"true", "t", "yes", "y", "1"}
#     vals = s.astype(str).str.strip().str.lower()
#     return vals.isin(truthy)
#
# def filter_by_pass_test_gate(df: pd.DataFrame) -> pd.DataFrame:
#     """Keep only rows where pass_test_gate is True (if present)."""
#     if "pass_test_gate" not in df.columns:
#         return df
#     mask = _coerce_bool_series(df["pass_test_gate"])
#     return df[mask].copy()
#
# def sort_and_dedup(df: pd.DataFrame) -> pd.DataFrame:
#     df = filter_by_pass_test_gate(df)
#
#     for col in ["test_precision", "test_accuracy", "n_preds_test", "val_precision"]:
#         if col in df.columns:
#             df[col] = pd.to_numeric(df[col], errors="coerce")
#
#     if "n_preds_test" in df.columns:
#         df = df[df["n_preds_test"] >= 100]
#
#     sort_cols = [c for c in ["test_precision", "test_accuracy", "n_preds_test", "val_precision"] if c in df.columns]
#     if sort_cols:
#         df = df.sort_values(by=sort_cols, ascending=[False] * len(sort_cols))
#
#     key_cols = [c for c in DEDUP_KEYS if c in df.columns]
#     if key_cols:
#         df = df.drop_duplicates(subset=key_cols, keep="first")
#     else:
#         df = df.drop_duplicates(keep="first")
#
#     return df.reset_index(drop=True)
#
# def combine_for_scoreline(scoreline: str, src_files: list[str], dest_files: list[str]) -> tuple[pd.DataFrame, list[str], list[str]]:
#     frames = []
#     consumed_src, consumed_dest = [], []
#
#     # Add source files
#     for p in src_files:
#         df = read_csv_safely(p)
#         if not df.empty:
#             df = filter_by_pass_test_gate(df)
#             if not df.empty:
#                 frames.append(df.assign(_source=p))
#                 consumed_src.append(p)
#             else:
#                 print(f"  [INFO] All rows failed pass_test_gate in source file: {p}")
#         else:
#             print(f"  [WARN] Empty source file (skipped): {p}")
#
#     # Add existing destination files for this scoreline (including prior COMBINEDs)
#     for p in dest_files:
#         df = read_csv_safely(p)
#         if not df.empty:
#             df = filter_by_pass_test_gate(df)
#             if not df.empty:
#                 frames.append(df.assign(_source=p))
#                 consumed_dest.append(p)
#             else:
#                 print(f"  [INFO] All rows failed pass_test_gate in dest file: {p}")
#         else:
#             print(f"  [WARN] Empty dest file (skipped): {p}")
#
#     if not frames:
#         return pd.DataFrame(), consumed_src, consumed_dest
#
#     combined = pd.concat(frames, ignore_index=True, sort=False)
#     combined = sort_and_dedup(combined)
#     return combined, consumed_src, consumed_dest
#
# def main():
#     # NEW: purge any *_FAILED files up-front in both locations
#     purge_failed_files([SRC_DIR, DEST_DIR])
#
#     # Map scorelines -> files in SRC and DEST (DEST now includes COMBINEDs)
#     src_map  = files_by_scoreline(SRC_DIR)
#     dest_map = files_by_scoreline(DEST_DIR)
#
#     all_scorelines = sorted(set(src_map) | set(dest_map))
#     if not all_scorelines:
#         print("No scoreline CSVs found in either directory; nothing to do.")
#         return
#
#     ts = datetime.now().strftime("%Y%m%d_%H%M%S")
#
#     for sl in all_scorelines:
#         src_files  = src_map.get(sl, [])
#         dest_files = dest_map.get(sl, [])
#
#         if not src_files and not dest_files:
#             continue
#
#         print(f"\n→ Processing scoreline: {sl} (src: {len(src_files)}, dest: {len(dest_files)})")
#         combined_df, consumed_src, consumed_dest = combine_for_scoreline(sl, src_files, dest_files)
#
#         if combined_df.empty:
#             print(f"  [WARN] Nothing to combine for {sl}.")
#             continue
#
#         # Write a fresh combined file to DEST
#         out_name = f"model_metrics_'{sl}'_COMBINED_{ts}.csv"
#         out_path = os.path.join(DEST_DIR, out_name)
#         combined_df.drop(columns=[c for c in ["_source"] if c in combined_df.columns], inplace=True, errors="ignore")
#         combined_df.to_csv(out_path, index=False)
#         print(f"  ✓ Wrote combined file with {len(combined_df)} rows:\n    {out_path}")
#
#         # Delete originals merged FROM SRC
#         for p in consumed_src:
#             try:
#                 os.remove(p)
#                 print(f"  ✂ Deleted source file: {p}")
#             except Exception as e:
#                 print(f"  [WARN] Could not delete source file {p}: {e}")
#
#         # Delete originals merged FROM DEST (older combineds/singles)
#         for p in consumed_dest:
#             if os.path.abspath(p) == os.path.abspath(out_path):
#                 continue
#             try:
#                 os.remove(p)
#                 print(f"  ✂ Deleted old dest file: {p}")
#             except Exception as e:
#                 print(f"  [WARN] Could not delete dest file {p}: {e}")
#
#     print("\nAll done.")
#
# if __name__ == "__main__":
#     main()


In [2]:
import os
import re
import glob
from datetime import datetime
import pandas as pd

# --- Paths ---
SRC_DIR  = r"C:\Users\leere\PycharmProjects\Football_ML3\Goals\2H_goal\ht_scoreline"
DEST_DIR = os.path.join(SRC_DIR, "best_models_by_ht_scoreline")
PATH_HT_SCORE = os.path.join(SRC_DIR, "path_ht_score")
os.makedirs(DEST_DIR, exist_ok=True)

# Safety for model-file pruning: set to False to actually delete non-best .pkl files
DRY_RUN_PKLS = False

# --- Filename patterns supported ---
PATTERNS = [
    re.compile(r"^model_metrics_'([^']+)'_(\d{8})_(\d{6})\.csv$"),
    re.compile(r"^model_metrics_\('([^']+)',\)_(\d{8})_(\d{6})\.csv$"),
    re.compile(r"^model_metrics_([^_]+)_(\d{8})_(\d{6})\.csv$"),
    re.compile(r"^model_metrics_'([^']+)'_COMBINED_(\d{8})_(\d{6})\.csv$"),
    re.compile(r"^model_metrics_\('([^']+)',\)_COMBINED_(\d{8})_(\d{6})\.csv$"),
    re.compile(r"^model_metrics_([^_]+)_COMBINED_(\d{8})_(\d{6})\.csv$"),
]

# Known parameter columns used for de-duplication
XGB_PARAM_COLS = [
    "n_estimators", "max_depth", "learning_rate", "min_child_weight",
    "subsample", "colsample_bytree", "reg_lambda",
]
MLP_PARAM_COLS = [
    "hidden_layer_sizes", "alpha", "learning_rate_init",
    "batch_size", "max_iter",
]
DEDUP_KEYS = XGB_PARAM_COLS + MLP_PARAM_COLS + ["threshold"]

# ---------- delete *_FAILED files ----------
def is_failed_filename(path: str) -> bool:
    base = os.path.basename(path)
    name_no_ext, _ = os.path.splitext(base)
    return base.endswith("_FAILED") or base.endswith("_FAILED.csv") or name_no_ext.endswith("_FAILED")

def purge_failed_files(dirs: list[str]) -> None:
    for d in dirs:
        pattern = os.path.join(d, "model_metrics_*")
        for p in glob.glob(pattern):
            if os.path.isfile(p) and is_failed_filename(p):
                try:
                    os.remove(p)
                    print(f"✂ Deleted FAILED file: {p}")
                except Exception as e:
                    print(f"[WARN] Could not delete FAILED file {p}: {e}")
# ------------------------------------------------

def extract_scoreline(filename: str) -> str | None:
    base = os.path.basename(filename)
    for rx in PATTERNS:
        m = rx.match(base)
        if m:
            return m.group(1)
    return None

def files_by_scoreline(base_dir: str) -> dict[str, list[str]]:
    out: dict[str, list[str]] = {}
    for path in glob.glob(os.path.join(base_dir, "model_metrics_*.csv")):
        if is_failed_filename(path):
            continue
        sl = extract_scoreline(os.path.basename(path))
        if not sl:
            continue
        out.setdefault(sl, []).append(path)
    return out

def read_csv_safely(path: str) -> pd.DataFrame:
    try:
        return pd.read_csv(path, low_memory=False)
    except Exception as e:
        print(f"[WARN] Failed to read {path}: {e}")
        return pd.DataFrame()

def _coerce_bool_series(s: pd.Series) -> pd.Series:
    if s.dtype == bool:
        return s
    try:
        sn = pd.to_numeric(s, errors="coerce")
        if sn.notna().any():
            return (sn.fillna(0) != 0)
    except Exception:
        pass
    truthy = {"true", "t", "yes", "y", "1"}
    vals = s.astype(str).str.strip().str.lower()
    return vals.isin(truthy)

def filter_by_pass_test_gate(df: pd.DataFrame) -> pd.DataFrame:
    if "pass_test_gate" not in df.columns:
        return df
    mask = _coerce_bool_series(df["pass_test_gate"])
    return df[mask].copy()

def sort_and_dedup(df: pd.DataFrame) -> pd.DataFrame:
    df = filter_by_pass_test_gate(df)

    # Coerce numeric for sorting metrics
    numeric_cols = ["val_precision_lcb", "val_precision", "test_precision", "test_accuracy", "n_preds_test"]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # Optional guard
    if "n_preds_test" in df.columns:
        df = df[df["n_preds_test"] >= 100]

    # Primary sort: val_precision_lcb (desc) with fallbacks if present
    sort_cols = [c for c in ["val_precision_lcb", "val_precision", "test_precision", "test_accuracy", "n_preds_test"] if c in df.columns]
    if sort_cols:
        df = df.sort_values(by=sort_cols, ascending=[False] * len(sort_cols))

    # De-duplicate on hyper-parameters (if available)
    key_cols = [c for c in DEDUP_KEYS if c in df.columns]
    if key_cols:
        df = df.drop_duplicates(subset=key_cols, keep="first")
    else:
        df = df.drop_duplicates(keep="first")

    return df.reset_index(drop=True)

def combine_for_scoreline(scoreline: str, src_files: list[str], dest_files: list[str]) -> tuple[pd.DataFrame, list[str], list[str]]:
    frames = []
    consumed_src, consumed_dest = [], []

    # Ingest source files
    for p in src_files:
        df = read_csv_safely(p)
        if not df.empty:
            df = filter_by_pass_test_gate(df)
            if not df.empty:
                frames.append(df.assign(_source=p))
                consumed_src.append(p)
            else:
                print(f"  [INFO] All rows failed pass_test_gate in source file: {p}")
        else:
            print(f"  [WARN] Empty source file (skipped): {p}")

    # Ingest existing dest files (older COMBINEDs/singles)
    for p in dest_files:
        df = read_csv_safely(p)
        if not df.empty:
            df = filter_by_pass_test_gate(df)
            if not df.empty:
                frames.append(df.assign(_source=p))
                consumed_dest.append(p)
            else:
                print(f"  [INFO] All rows failed pass_test_gate in dest file: {p}")
        else:
            print(f"  [WARN] Empty dest file (skipped): {p}")

    if not frames:
        return pd.DataFrame(), consumed_src, consumed_dest

    combined = pd.concat(frames, ignore_index=True, sort=False)
    combined = sort_and_dedup(combined)
    return combined, consumed_src, consumed_dest

# ---------- gather best model_pkl from COMBINEDs ----------
def _norm_path(p: str) -> str | None:
    if not isinstance(p, str) or not p.strip():
        return None
    p = p.strip().strip('"').strip("'")
    if not os.path.isabs(p):
        p = os.path.join(PATH_HT_SCORE, p)
    try:
        return os.path.realpath(p)
    except Exception:
        return os.path.abspath(p)

def collect_best_model_pkls_from_combined() -> set[str]:
    keep = set()
    combined_glob = os.path.join(DEST_DIR, "model_metrics_*_COMBINED_*.csv")
    files = sorted(glob.glob(combined_glob))
    if not files:
        print("[INFO] No COMBINED files found to collate best models from.")
        return keep

    for f in files:
        df = read_csv_safely(f)
        if df.empty:
            continue
        # Ensure current sorting rule is applied
        df = sort_and_dedup(df)

        if "model_pkl" not in df.columns or df.empty:
            if "model_pkl" not in df.columns:
                print(f"  [WARN] 'model_pkl' column missing in {f}; skipping.")
            continue

        best_path_raw = str(df.iloc[0]["model_pkl"])
        np = _norm_path(best_path_raw)
        if np:
            keep.add(np)
            print(f"  ✓ Best model from {os.path.basename(f)} -> {np}")
        else:
            print(f"  [WARN] Could not normalise model path from {f}: {best_path_raw}")

    return keep

# ---------- prune PATH_HT_SCORE to only keep best ----------
def prune_path_ht_score(keep_paths: set[str]) -> None:
    if not os.path.isdir(PATH_HT_SCORE):
        print(f"[WARN] PATH_HT_SCORE does not exist: {PATH_HT_SCORE}")
        return

    all_files = [os.path.realpath(os.path.join(PATH_HT_SCORE, name))
                 for name in os.listdir(PATH_HT_SCORE)
                 if os.path.isfile(os.path.join(PATH_HT_SCORE, name))]

    all_pkls = [p for p in all_files if p.lower().endswith(".pkl")]
    keep_pkls = {p for p in keep_paths if p.lower().endswith(".pkl")}

    to_delete = sorted(set(all_pkls) - keep_pkls)

    print("\n— Pruning path_ht_score —")
    print(f" Found .pkl files: {len(all_pkls)}")
    print(f" Best .pkl (keep): {len(keep_pkls)}")
    print(f" Will delete: {len(to_delete)}")

    if DRY_RUN_PKLS:
        for p in to_delete[:20]:
            print(f"   (dry-run) would delete: {p}")
        if len(to_delete) > 20:
            print(f"   ...and {len(to_delete) - 20} more.")
        print(" Set DRY_RUN_PKLS = False to actually delete.")
        return

    for p in to_delete:
        try:
            os.remove(p)
            print(f" ✂ Deleted: {p}")
        except Exception as e:
            print(f" [WARN] Could not delete {p}: {e}")

# -----------------------------

def main():
    # 1) Clean up *_FAILED CSVs first
    purge_failed_files([SRC_DIR, DEST_DIR])

    # 2) Map files by scoreline (SRC and DEST), so we can combine
    src_map  = files_by_scoreline(SRC_DIR)
    dest_map = files_by_scoreline(DEST_DIR)

    all_scorelines = sorted(set(src_map) | set(dest_map))
    if not all_scorelines:
        print("No scoreline CSVs found in either directory; nothing to do.")
    else:
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")

        for sl in all_scorelines:
            src_files  = src_map.get(sl, [])
            dest_files = dest_map.get(sl, [])

            if not src_files and not dest_files:
                continue

            print(f"\n→ Processing scoreline: {sl} (src: {len(src_files)}, dest: {len(dest_files)})")
            combined_df, consumed_src, consumed_dest = combine_for_scoreline(sl, src_files, dest_files)

            if combined_df.empty:
                print(f"  [WARN] Nothing to combine for {sl}.")
                # If nothing combined, still remove any empty/failed leftovers from SRC only
                continue

            # Write fresh COMBINED to DEST
            out_name = f"model_metrics_'{sl}'_COMBINED_{ts}.csv"
            out_path = os.path.join(DEST_DIR, out_name)
            combined_df.drop(columns=[c for c in ["_source"] if c in combined_df.columns], inplace=True, errors="ignore")
            combined_df.to_csv(out_path, index=False)
            print(f"  ✓ Wrote combined file with {len(combined_df)} rows:\n    {out_path}")

            # Delete originals merged FROM SRC
            for p in consumed_src:
                try:
                    os.remove(p)
                    print(f"  ✂ Deleted source file: {p}")
                except Exception as e:
                    print(f"  [WARN] Could not delete source file {p}: {e}")

            # Delete originals merged FROM DEST (older COMBINEDs/singles), but never the brand-new file
            for p in consumed_dest:
                if os.path.abspath(p) == os.path.abspath(out_path):
                    continue
                try:
                    os.remove(p)
                    print(f"  ✂ Deleted old dest file: {p}")
                except Exception as e:
                    print(f"  [WARN] Could not delete dest file {p}: {e}")

    # 3) Collate best model files (top-row 'model_pkl') from the remaining COMBINEDs (the fresh ones)
    keep_best_pkls = collect_best_model_pkls_from_combined()

    # 4) Prune path_ht_score to only those best models
    prune_path_ht_score(keep_best_pkls)

    print("\nAll done.")

if __name__ == "__main__":
    main()


✂ Deleted FAILED file: C:\Users\leere\PycharmProjects\Football_ML3\Goals\2H_goal\ht_scoreline\model_metrics_'2-0'_20250818_105249_FAILED.csv
✂ Deleted FAILED file: C:\Users\leere\PycharmProjects\Football_ML3\Goals\2H_goal\ht_scoreline\model_metrics_'3-0'_20250818_103925_FAILED.csv

→ Processing scoreline: 0-0 (src: 1, dest: 1)
  ✓ Wrote combined file with 50 rows:
    C:\Users\leere\PycharmProjects\Football_ML3\Goals\2H_goal\ht_scoreline\best_models_by_ht_scoreline\model_metrics_'0-0'_COMBINED_20250818_125945.csv
  ✂ Deleted source file: C:\Users\leere\PycharmProjects\Football_ML3\Goals\2H_goal\ht_scoreline\model_metrics_'0-0'_20250818_094805.csv
  ✂ Deleted old dest file: C:\Users\leere\PycharmProjects\Football_ML3\Goals\2H_goal\ht_scoreline\best_models_by_ht_scoreline\model_metrics_'0-0'_COMBINED_20250816_113140.csv

→ Processing scoreline: 0-1 (src: 1, dest: 1)
  ✓ Wrote combined file with 41 rows:
    C:\Users\leere\PycharmProjects\Football_ML3\Goals\2H_goal\ht_scoreline\best_model